In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import os

root_dir = '../archive/'
train_dir = root_dir + 'train/'
test_dir = root_dir + 'test/'
train_files = os.listdir(train_dir)
test_files = os.listdir(test_dir)

for item in range(len(train_files)-1):
    if train_files[item].startswith('.'):
        train_files.pop(item)
    if test_files[item].startswith('.'):
        test_files.pop(item)
        
print('train_files --',train_files)
print('test_files  --',test_files)

train_files -- ['happy', 'sad', 'fearful', 'neutral', 'angry', 'disgusted', 'surprised']
test_files  -- ['happy', 'sad', 'fearful', 'neutral', 'angry', 'disgusted', 'surprised']


In [38]:
!pip install imageio

     |████████████████████████████████| 3.3 MB 8.3 MB/s eta 0:00:01


In [50]:
#samples from each category of emotion and puts them in a dictionary for test and train set

def get_images(train_dir, test_dir):
    train_emo = {}
    test_emo = {}
    for emotion in train_files:
        path = train_dir + '{}/'.format(emotion)
        files = os.listdir(path)
        train_emo[emotion] = files[:20]
    for emotion in test_files:
        path = test_dir + '{}/'.format(emotion)
        files = os.listdir(path)
        test_emo[emotion] = files[:20]
    return test_emo, train_emo

tr, ts = get_images(train_dir, test_dir) #dictionaries of image paths with labels 

tr.keys()
#def png_to_np(dictionary):
    


#im = imageio.imread(train_dir+'surprised/im2714.png')
    
im.flatten()


Array([254, 255, 249, ..., 141, 151,  80], dtype=uint8)